# Mass Spring Systems with Explicit Euler Integration Scheme

Using explicit Euler integration schemes to simulate mass spring systems, with and without damping

Author: Andres Ibarra, written as part of 22F Leave Term Research

Credit to professor Bo Zhu, on whose COSC89 Course Notes the following math is based

## 2 particle system

To remove damping, change line 59 to be `drag_damping[None] = 0.`

Each particle's mass can be controlled individually using keyboard inputs

In [8]:
import taichi as ti

ti.init()

num_particles = 2
dt = 1e-3
substeps = 10

spring_k = ti.field(dtype=ti.f32, shape=())
drag_damping = ti.field(dtype=ti.f32, shape=())
mass = ti.field(dtype=ti.f32, shape=num_particles)

x = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
rest_length = ti.field(dtype=ti.f32, shape=())


@ti.kernel
def substep():
    
    # find force
    for i in range(num_particles):
        f[i] = [0,0] # reset force, since we're calculating it from scratch each time
        
        for j in range(num_particles):
            if i != j:
                x_ij = x[j] - x[i]
                d = x_ij.norm() # euclidean distance

                # spring force on i by j
                f[i] +=  spring_k[None] * x_ij / d * (d - rest_length[None])
                # damping force
                f[i] += drag_damping[None] * (v[j] - v[i]).dot(x_ij / d) * (x_ij / d)
        
    for i in range(num_particles):
        # update v
        v[i] = v[i] + dt * f[i] / mass[i]
        # update x
        x[i] = x[i] + dt * v[i]
    

def main():
    gui = ti.GUI('Two Springs with damping', (640, 480))
    
    # initialize positions of particles
    x[0][0] = 0.7 # left x
    x[0][1] = 0.5 # left y
    x[1][0] = 0.3 # right x
    x[1][1] = 0.5 # right y
    # set initial velocities to 0
    v[0][0] = 0
    v[0][1] = 0
    v[1][0] = 0
    v[1][1] = 0
    
    rest_length[None] = 0.3
    spring_k[None] = 10
    drag_damping[None] = 0.3
    for i in range(num_particles):
        mass[i] = 0.5
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == 'a': # lower left mass 
                mass[0] /= 1.1
            elif e.key == 'd': # increase left mass
                mass[0] *= 1.1
            elif e.key == 's': # lower right mass
                mass[1] /= 1.1
            elif e.key == 'w': # increase right mass
                mass[1] *= 1.1
            
        # Move stuff
        for step in range(substeps):
            substep()
        
        # Draw spring
        X = x.to_numpy()
        gui.line(begin=X[0], end=X[1], radius = 2, color = 0x444444)        
        # Draw particles
        for i in range(num_particles):
            gui.circle(pos=X[i], radius = 5)
        
        # show the current values
        gui.text(
            content='Press a to decrease the left mass, d to increase it',
            pos=(0,0.99),
            color=0xffffff)
        gui.text(
            content='Press s to decrease the right mass, w to increase it',
            pos=(0,0.95),
            color=0xffffff)
        gui.text(content=f'Left Mass {mass[0]:.1f}',
                 pos=(0, 0.90),
                 color=0xffffff)
        gui.text(content=f'Right Mass {mass[1]:.1f}',
                 pos=(0, 0.85),
                 color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()



[Taichi] Starting on arch=x64


## Multi particle system (fully connected)

Uses mouse clicks to add new particles, automatically adding springs to keep the system fully connected. Keyboard inputs can also be used to modify the spring constan. Damping is deactivated for this, but can be added by uncommenting line 35

In [2]:
import taichi as ti

ti.init()

# constants
max_particles = 10
num_particles = ti.field(dtype=ti.i32, shape=())
dt = 1e-3
substeps = 10

# globals
spring_k = ti.field(dtype=ti.f32, shape=())
drag_damping = ti.field(dtype=ti.f32, shape=())
mass = ti.field(dtype=ti.f32, shape=max_particles)

x = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
rest_length = ti.field(dtype=ti.f32, shape=())


@ti.kernel
def substep():
    
    # find force
    for i in range(num_particles[None]):
        f[i] = [0,0] # reset force, since we're calculating it from scratch each time
        
        for j in range(num_particles[None]):
            if i != j:
                x_ij = x[j] - x[i]
                d = x_ij.norm() # euclidean distance

                # spring force on i by j
                f[i] +=  spring_k[None] * x_ij / d * (d - rest_length[None])
                # damping force
                # f[i] += drag_damping[None] * (v[j] - v[i]).dot(x_ij / d) * (x_ij / d)
        
    for i in range(num_particles[None]):
        # update v
        v[i] = v[i] + dt * f[i] / mass[i]
        # update x
        x[i] = x[i] + dt * v[i]

        
@ti.kernel
def new_particle(pos_x: ti.f32, pos_y: ti.f32):
    particle_id = num_particles[None]
    if particle_id < max_particles:
        x[particle_id] = [pos_x, pos_y]
        v[particle_id] = [0, 0]
        num_particles[None] += 1
        

def main():
    gui = ti.GUI('Multi Particle Mass-Spring System', (640, 480))
    
    new_particle(0.3, 0.5)
    
    # initialize values
    rest_length[None] = 0.3
    spring_k[None] = 10
    drag_damping[None] = 0.3
    for i in range(max_particles):
        mass[i] = 0.5
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.UP:
                spring_k[None] *= 1.1
            elif e.key == ti.GUI.DOWN:
                spring_k[None] /= 1.1
            elif e.key == ti.GUI.LMB:
                new_particle(e.pos[0], e.pos[1])
            
        # Move stuff
        for step in range(substeps):
            substep()
        
        # Draw springs
        X = x.to_numpy()
        for i in range(num_particles[None]):
            for j in range(num_particles[None]):
                if i != j:
                    gui.line(begin=X[i], end=X[j], radius = 2, color = 0x444444)        
        # Draw particles
        for i in range(num_particles[None]):
            gui.circle(pos=X[i], radius = 5)
        
        # show the current values
        gui.text(
            content='Click anywhere on screen to add a new mass',
            pos=(0,0.99),
            color=0xffffff)
        gui.text(
            content='Press up/down arrows to change spring constant',
            pos=(0,0.95),
            color=0xffffff)
        gui.text(content=f'k {spring_k[None]:.1f}',
                 pos=(0, 0.90),
                 color=0xffffff)
        gui.text(content=f'Num particles {num_particles[None]}',
                 pos=(0, 0.85),
                 color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64
